# Text Translation and Sentiment Analysis using Transformers

## Project Overview:

The objective of this project is to analyze the sentiment of movie reviews in three different languages - English, French, and Spanish. We have been given 30 movies, 10 in each language, along with their reviews and synopses in separate CSV files named `movie_reviews_eng.csv`, `movie_reviews_fr.csv`, and `movie_reviews_sp.csv`.

- The first step of this project is to convert the French and Spanish reviews and synopses into English. This will allow us to analyze the sentiment of all reviews in the same language. We will be using pre-trained transformers from HuggingFace to achieve this task.

- Once the translations are complete, we will create a single dataframe that contains all the movies along with their reviews, synopses, and year of release in all three languages. This dataframe will be used to perform sentiment analysis on the reviews of each movie.

- Finally, we will use pretrained transformers from HuggingFace to analyze the sentiment of each review. The sentiment analysis results will be added to the dataframe. The final dataframe will have 30 rows


The output of the project will be a CSV file with a header row that includes column names such as **Title**, **Year**, **Synopsis**, **Review**, **Review Sentiment**, and **Original Language**. The **Original Language** column will indicate the language of the review and synopsis (*en/fr/sp*) before translation. The dataframe will consist of 30 rows, with each row corresponding to a movie.

In [1]:
# imports
import pandas as pd
from transformers import MarianMTModel, MarianTokenizer
from transformers import pipeline
import numpy as np

/opt/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Get data from  `.csv` files and then preprocess data

In [2]:
# use the `pd.read_csv()` function to read the movie_review_*.csv files into 3 separate pandas dataframes

# Note: All the dataframes would have different column names. For testing purposes
# you should have the following column names/headers -> [Title, Year, Synopsis, Review]

def preprocess_data() -> pd.DataFrame:
    """
    Reads movie data from .csv files, map column names, add the "Original Language" column,
    and finally concatenate in one resultant dataframe called "df".
    """
  
    df_fr = pd.read_csv("data/movie_reviews_fr.csv", header = None, names = ['Title', 'Year', 'Synopsis', 'Review'])
    df_sp = pd.read_csv("data/movie_reviews_sp.csv", header = None, names =['Title', 'Year', 'Synopsis', 'Review'])
    df_eng = pd.read_csv("data/movie_reviews_eng.csv", header = None, names =['Title', 'Year', 'Synopsis', 'Review'])
    df_fr.insert(4, 'Language', 'French')
    df_sp.insert(4, 'Language', 'Spanish')
    df_eng.insert(4, 'Language', 'English')
    frames = [df_fr, df_sp, df_eng]
    df = pd.concat(frames)

    #TODO 1: Add your code here
    return df

df = preprocess_data()
df.tail()

,Title,Year,Synopsis,Review,Language
6,Blade Runner 2049,2017,"Officer K (Ryan Gosling), a new blade runner f...","""Boring and too long. Nothing like the origina...",English
7,Scott Pilgrim vs. the World,2010,Scott Pilgrim (Michael Cera) must defeat his n...,"""It was difficult to sit through the whole thi...",English
8,The Nice Guys,2016,"In 1970s Los Angeles, a private eye (Ryan Gosl...","""The Nice Guys tries too hard to be funny, and...",English
9,Solo: A Star Wars Story,2018,A young Han Solo (Alden Ehrenreich) joins a gr...,"""Dull and pointless, with none of the magic of...",English
10,The Island,2005,In a future where people are cloned for organ ...,"""The Island is a bland and forgettable sci-fi ...",English


In [3]:
df.sample(10)

,Title,Year,Synopsis,Review,Language
1,Roma,2018,Cleo (Yalitza Aparicio) es una joven empleada ...,"""Roma es una película hermosa y conmovedora qu...",Spanish
10,The Island,2005,In a future where people are cloned for organ ...,"""The Island is a bland and forgettable sci-fi ...",English
0,Titre,Année,Synopsis,Critiques,French
5,Amores perros,2000,Tres historias se entrelazan en esta película ...,"""Amores perros es una película intensa y conmo...",Spanish
3,Forrest Gump,1994,Forrest Gump (Tom Hanks) is a simple man with ...,"""Forrest Gump is a heartwarming and inspiratio...",English
2,Intouchables,2011,Ce film raconte l'histoire de l'amitié improba...,"""Intouchables est un film incroyablement touch...",French
10,Babylon A.D.,2008,"Dans un futur lointain, un mercenaire doit esc...","""Ce film est un gâchis complet. Les personnage...",French
2,The Dark Knight,2008,Batman (Christian Bale) teams up with District...,"""The Dark Knight is a thrilling and intense su...",English
7,Scott Pilgrim vs. the World,2010,Scott Pilgrim (Michael Cera) must defeat his n...,"""It was difficult to sit through the whole thi...",English
0,Movie / TV Series,Year,Synopsis,Review,English


### Text translation

Translate the **Review** and **Synopsis** column values to English.

In [4]:
# load translation models and tokenizers
# TODO 2: Update the code below
fr_en_model_name = "Helsinki-NLP/opus-mt-fr-en"
es_en_model_name = "Helsinki-NLP/opus-mt-es-en"
fr_en_model = MarianMTModel.from_pretrained(fr_en_model_name) 
es_en_model = MarianMTModel.from_pretrained(es_en_model_name)
fr_en_tokenizer = MarianTokenizer.from_pretrained(fr_en_model_name)
es_en_tokenizer = MarianTokenizer.from_pretrained(es_en_model_name)

# TODO 3: Complete the function below
def translate(text: str, model, tokenizer) -> str:
    """
    function to translate a text using a model and tokenizer
    """
    # encode the text using the tokenizer
    inputs = tokenizer(text, return_tensors = 'pt')

    # generate the translation using the model
    outputs = model.generate(**inputs)

    # decode the generated output and return the translated text
    decoded = tokenizer.batch_decode(outputs[0])
    return decoded

/opt/venv/lib/python3.10/site-packages/transformers/models/marian/tokenization_marian.py:194: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [5]:
# TODO 4: Update the code below

# filter reviews in French and translate to English
fr_reviews = df.loc[df['Language'] == 'French', 'Review']
fr_reviews_en =  [translate(i,fr_en_model,fr_en_tokenizer) for i in fr_reviews]
 
# filter synopsis in French and translate to English
fr_synopsis = df.loc[df['Language'] == 'French', 'Synopsis']
fr_synopsis_en = [translate(i,fr_en_model,fr_en_tokenizer) for i in fr_synopsis]
 

# filter reviews in Spanish and translate to English
es_reviews = df.loc[df['Language'] == 'Spanish', 'Review']
es_reviews_en = [translate(i,es_en_model,es_en_tokenizer) for i in es_reviews]

# filter synopsis in Spanish and translate to English
es_synopsis = df.loc[df['Language'] == 'Spanish', 'Synopsis']
es_synopsis_en = [translate(i,es_en_model,es_en_tokenizer) for i in es_synopsis]



    


/opt/venv/lib/python3.10/site-packages/transformers/generation/utils.py:1288: UserWarning: Using `max_length`'s default (512) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [6]:


df.loc[df['Language'] == 'French', 'Review'] = fr_reviews_en 

df.loc[df['Language'] == 'Spanish', 'Review'] = es_reviews_en 
    
    
df.loc[df['Language'] == 'French', 'Synopsis'] = fr_synopsis_en 

df.loc[df['Language'] == 'Spanish', 'Synopsis'] = es_synopsis_en 
    
   



In [7]:
df.sample(10)

,Title,Year,Synopsis,Review,Language
10,The Island,2005,In a future where people are cloned for organ ...,"""The Island is a bland and forgettable sci-fi ...",English
10,Babylon A.D.,2008,"[<pad>, In, the, distant, future, ,, a, mercen...","[<pad>, "", This, film, is, a, complete, mess, ...",French
0,Título,Año,"[<pad>, Overview, </s>]","[<pad>, C, ritic, ism, s, </s>]",Spanish
5,Le Fabuleux Destin d'Amélie Poulain,2001,"[<pad>, This, romantic, comedy, tells, the, st...","[<pad>, "", The, , Fabulous, Des, tin, y, of, A...",French
8,The Nice Guys,2016,"In 1970s Los Angeles, a private eye (Ryan Gosl...","""The Nice Guys tries too hard to be funny, and...",English
7,La Tour Montparnasse Infernale,2001,"[<pad>, Two, in, competent, office, workers, f...","[<pad>, "", I, can, ', t, believe, I, ', ve, wa...",French
1,Roma,2018,"[<pad>, Cle, o, (, Y, a, litz, a, A, par, icio...","[<pad>, "", Rom, e, is, a, beautiful, and, movi...",Spanish
5,Amores perros,2000,"[<pad>, Three, stories, inter, t, win, e, in, ...","[<pad>, "", A, more, s, dogs, is, an, intense, ...",Spanish
6,Águila Roja,(2009-2016),"[<pad>, This, Spanish, television, series, fol...","[<pad>, "", R, ed, Eagle, is, a, boring, and, u...",Spanish
6,Blade Runner 2049,2017,"Officer K (Ryan Gosling), a new blade runner f...","""Boring and too long. Nothing like the origina...",English


### Sentiment Analysis

Use HuggingFace pretrained model for sentiment analysis of the reviews. Store the sentiment result **Positive** or **Negative** in a new column titled **Sentiment** in the dataframe.

In [8]:
# TODO 5: Update the code below
# load sentiment analysis model
model_name = "distilbert-base-uncased-finetuned-sst-2-english"
sentiment_classifier = pipeline(model=model_name)

# TODO 6: Complete the function below
def analyze_sentiment(text, classifier):
    """
    function to perform sentiment analysis on a text using a model
    """


   
    outputs = classifier(text)

    return outputs

In [9]:
# TODO 7: Add code below for sentiment analysis
# perform sentiment analysis on reviews and store results in new column
sentiment = [analyze_sentiment(i, sentiment_classifier) for i in df['Review']]



In [10]:
df['Sentiment']= sentiment

In [12]:
df.sample(10)

,Title,Year,Synopsis,Review,Language,Sentiment
8,The Nice Guys,2016,"In 1970s Los Angeles, a private eye (Ryan Gosl...","""The Nice Guys tries too hard to be funny, and...",English,"[{'label': 'NEGATIVE', 'score': 0.999802410602..."
8,Astérix aux Jeux Olympiques,2008,"[<pad>, In, this, film, adaptation, of, the, p...","[<pad>, "", This, film, is, a, complete, disapp...",French,"[{'label': 'NEGATIVE', 'score': 0.725283205509..."
0,Título,Año,"[<pad>, Overview, </s>]","[<pad>, C, ritic, ism, s, </s>]",Spanish,"[{'label': 'NEGATIVE', 'score': 0.725283205509..."
3,Y tu mamá también,2001,"[<pad>, Two, teenage, friends, (, Ga, el, Garc...","[<pad>, "", And, your, mom, is, also, a, movie,...",Spanish,"[{'label': 'NEGATIVE', 'score': 0.725283205509..."
2,Intouchables,2011,"[<pad>, This, film, tells, the, story, of, the...","[<pad>, "", Un, touch, able, s, is, an, incredi...",French,"[{'label': 'NEGATIVE', 'score': 0.725283205509..."
10,The Island,2005,In a future where people are cloned for organ ...,"""The Island is a bland and forgettable sci-fi ...",English,"[{'label': 'NEGATIVE', 'score': 0.999816358089..."
6,Le Dîner de Cons,1998,"[<pad>, The, film, follows, the, story, of, a,...","[<pad>, "", I, didn, ', t, like, this, movie, a...",French,"[{'label': 'NEGATIVE', 'score': 0.725283205509..."
1,The Shawshank Redemption,1994,"Andy Dufresne (Tim Robbins), a successful bank...","""The Shawshank Redemption is an inspiring tale...",English,"[{'label': 'POSITIVE', 'score': 0.999888062477..."
6,Blade Runner 2049,2017,"Officer K (Ryan Gosling), a new blade runner f...","""Boring and too long. Nothing like the origina...",English,"[{'label': 'NEGATIVE', 'score': 0.999675154685..."
5,Le Fabuleux Destin d'Amélie Poulain,2001,"[<pad>, This, romantic, comedy, tells, the, st...","[<pad>, "", The, , Fabulous, Des, tin, y, of, A...",French,"[{'label': 'NEGATIVE', 'score': 0.725283205509..."


In [14]:
# export the results to a .csv file
df.to_csv("reviews_with_sentiment.csv", index=False)